In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append("/Users/emrebaloglu/Documents/RL/basic_reinforcement_learning")
from RL_for_NLP.text_environments import TextEnvClfBert, TextEnvClf
from RL_for_NLP.text_data_pools import PartialReadingDataPoolWithWord2Vec, PartialReadingDataPoolWithBertTokens
import NLP_utils.preprocessing as nlp_processing
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
import json

with open("/Users/emrebaloglu/Documents/RL/basic_reinforcement_learning/NLP_datasets/RT_Polarity/data_info_bert.json", "r") as f:
    data_info = json.load(f)
data_info

{'path': '/Users/emrebaloglu/Documents/RL/basic_reinforcement_learning/NLP_datasets/RT_Polarity',
 'max_len': 50,
 'vocab_size': 28996}

In [3]:
data_train = nlp_processing.openDfFromPickle(data_info["path"] + "/rt-polarity-train-bert.pkl")
data_train.head()

,Unnamed: 0,label,review,label_str,review_bert_input_ids,review_bert_token_type_ids,review_bert_attention_mask
0,5829,1,does what fine documentary does best it extend...,good,"[101, 1674, 1184, 2503, 4148, 1674, 1436, 1122...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,3209,0,it drowns in sap,bad,"[101, 1122, 22592, 1116, 1107, 21718, 1643, 10...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
2,4323,0,the scriptwriters are no less menace to societ...,bad,"[101, 1103, 5444, 13814, 1116, 1132, 1185, 175...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,281,0,there an excellent minute film here unfortunat...,bad,"[101, 1175, 1126, 6548, 2517, 1273, 1303, 1667...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ..."
4,100,0,narratively trouble every day is plodding mess,bad,"[101, 8195, 1193, 3819, 1451, 1285, 1110, 185,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ..."


In [4]:
# declare some hyperparameters

WINDOW_SIZE = 8
MAX_STEPS = int(1e+5)

In [5]:
data_test = nlp_processing.openDfFromPickle(data_info["path"] + "/rt-polarity-test-bert.pkl")
data_val = nlp_processing.openDfFromPickle(data_info["path"] + "/rt-polarity-val-bert.pkl")

train_pool = PartialReadingDataPoolWithBertTokens(data_train, "review", "label", "good", WINDOW_SIZE)
test_pool = PartialReadingDataPoolWithBertTokens(data_test, "review", "label", "good", WINDOW_SIZE)
val_pool = PartialReadingDataPoolWithBertTokens(data_val, "review", "label", "good", WINDOW_SIZE)

Maximum sentence length in pool: 50
Maximum sentence length in pool: 50
Maximum sentence length in pool: 50


In [6]:
train_env = TextEnvClfBert(train_pool, MAX_STEPS)
val_env = TextEnvClfBert(val_pool, 1000)
test_env = TextEnvClfBert(test_pool, 1000)

In [7]:
train_env.current_observation

BertObservation(sample_str='does what fine documentary does best it extends warm invitation into an unfamiliar world then illuminates it fully and allows the larger implications of the journey to sink in unobtrusively ', sample_input_id_vecs=[array([ 101., 1674., 1184., 2503., 4148., 1674., 1436., 1122.]), array([ 8559.,  3258.,  8727.,  1154.,  1126., 17546.,  1362.,  1173.]), array([ 5178., 14088., 13978.,  1116.,  1122.,  3106.,  1105.,  3643.]), array([ 1103.,  2610., 14755.,  1104.,  1103.,  5012.,  1106.,  7496.]), array([ 1107.,  8362., 12809., 18062., 17849.,  1193.,   102.,     0.]), array([0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0.])], sample_token_type_vecs=[array([0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0.])], sampl

In [8]:
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import reinforce_algorithm_utils as rl_monte_carlo
import gym
import torch
from torch.optim import Adam

import imageio

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}.")

Using device: cpu.


In [10]:
print(train_env.current_state_input_id.shape)
s_size = train_env.current_state_input_id.shape[0]
a_size = len(train_env.action_space)
print(s_size, a_size)

(8,)
8 4


In [11]:
hyperparameters = {
    "h_sizes": [64, 32],
    "n_training_episodes": 100000,
    "n_evaluation_episodes": 10,
    "max_t": 25,
    "gamma": 1.0,
    "lr": 1e-2,
    "env_id": None,
    "state_space": s_size,
    "action_space": a_size,
}

In [12]:
# Create policy and place it to the device
import policy_networks as pn
"""policy = pn.Transformer_Baseline_Policy(data_info["vocab_size"], hyperparameters["state_space"], hyperparameters["action_space"],
                                        num_heads=1, num_layers=1)"""
policy = pn.RNN_Baseline_Policy(data_info["vocab_size"], hyperparameters["state_space"], hyperparameters["action_space"])
optimizer = Adam(policy.parameters(), lr=hyperparameters["lr"])

In [51]:
params = dict(policy.named_modules())
print(params)


{'': RNN_Baseline_Policy(
  (embed_enc): Embedding(28996, 5, max_norm=True)
  (rnn): GRU(5, 2, num_layers=2, batch_first=True, bidirectional=True)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=32, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=4, bias=True)
), 'embed_enc': Embedding(28996, 5, max_norm=True), 'rnn': GRU(5, 2, num_layers=2, batch_first=True, bidirectional=True), 'flat': Flatten(start_dim=1, end_dim=-1), 'fc1': Linear(in_features=32, out_features=50, bias=True), 'fc2': Linear(in_features=50, out_features=50, bias=True), 'fc3': Linear(in_features=50, out_features=4, bias=True)}


In [55]:
import mlflow
import json

params_s = str(params)
mlflow.log_dict(params_s, "params.json")

In [ ]:
scores = rl_monte_carlo.reinforce_algorithm(train_env, policy,
                   optimizer,
                   hyperparameters["n_training_episodes"], 
                   hyperparameters["max_t"],
                   hyperparameters["gamma"])

In [58]:
torch.arange(1, 8)

tensor([1, 2, 3, 4, 5, 6, 7])

In [75]:
-1e-1*(np.log2(500000))

-1.8931568569324175

In [ ]:
rl_monte_carlo.evaluate_agent(train_env, 10, 100, policy)

In [ ]:
rl_monte_carlo.evaluate_agent(val_env, 10, 100, policy)

In [ ]:
rl_monte_carlo.evaluate_on_clf(val_env, policy, pos_label="good")

In [ ]:
def eval_model(model, env):
    done = False
    obs = env.reset()
    total_reward = 0.0
    actions = []
    while not done:
        action, _states = model.predict(obs)
        action = action.item()
        obs, rewards, done, info = env.step(action)
        actions.append(env.action_space.ix_to_action(action))
        total_reward += rewards
    print("---------------------------------------------")
    print(f"Predicted Label {actions}")
    print(f"Oracle Label: {env.current_label}")
    print(f"Total Reward: {total_reward}")
    print("---------------------------------------------")


In [ ]:
eval_model(policy, val_env)

In [ ]:
import mlflow

In [ ]:

mlflow.set_tracking_uri("file:/Users/emrebaloglu/Documents/RL/basic_reinforcement_learning/agent_models/rt-polarity")
mlflow.set_experiment("rnn")
mlflow.start_run()
mlflow.log_param("n_episodes", 500)
mlflow.pytorch.log_model(policy, "rnn-default")
mlflow.end_run()

In [ ]:
mlflow.end_run()

In [ ]:
mlflow.set_tracking_uri("file:/Users/emrebaloglu/Documents/RL/basic_reinforcement_learning/agent_models/rt-polarity")
uri = "runs:/9f09e3fa6df6427ba2889e89ac787bcc/rnn-default"

model = mlflow.pytorch.load_model(uri)

In [ ]:
rl_monte_carlo.evaluate_agent(env, 10, 100, model)

In [ ]:
rl_monte_carlo.evaluate_on_clf(env, model)

In [ ]:
from stable_baselines3.dqn.policies import MlpPolicy
from stable_baselines3 import DQN

model = DQN(policy=MlpPolicy, env=train_env, learning_rate=0.001, batch_size=3)

for i in range(int(5)):
    model.learn(total_timesteps=int(1e+3), reset_num_timesteps=False)
    eval_model(model, val_env)

In [ ]:
from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
# Get the state space and action space
s_size = train_env.observation_space.shape[0]
a_size = len(train_env.action_space.actions)
print(s_size, a_size)

In [ ]:
model = A2C(policy = "MlpPolicy",
            env = train_env,
            gae_lambda = 0.9,
            gamma = 0.99,
            learning_rate = 0.00096,
            max_grad_norm = 0.5,
            n_steps = 8,
            vf_coef = 0.4,
            ent_coef = 0.0,
            tensorboard_log = "./tensorboard",
            policy_kwargs=dict(
            log_std_init=-2, ortho_init=False),
            normalize_advantage=False,
            use_rms_prop= True,
            verbose=1)

In [ ]:
for i in range(3):
    model.learn(500, log_interval=250)
    print(evaluate_policy(model, val_env, 3))

In [ ]:
evaluate_policy(model, val_env, n_eval_episodes=1)

In [ ]:
rl_monte_carlo.evaluate_on_clf(val_env, model)